# CoursewareHubのセットアップ--直接学認フェデレーションを利用する

---

VCノードにCoursewareHub環境を構築します。

## 概要

CoursewareHubの構成要素を以下に示します。

![構成](images/cw-321-01.png)


このNotebookでは上図で `CoursewareHub` を示す枠内にあるものを構築します。


> このNotebookでは IdP-proxyを利用せずに、直接CoursewareHubを学認のSPとして登録する手順について記しています。

各コンテナの役割や処理の詳細については https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy を参照してください。

### 事前に準備が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* CoursewareHubのサーバ証明書
* 学認クラウドゲートウェイのグループ名

#### CoursewareHubのサーバ証明書

CoursewareHubではHTTPSによる公開を行うためサーバ証明書とその秘密鍵を準備する必要があります。また必要に応じて中間CA証明書を準備してください。

#### 学認クラウドゲートウェイのグループ

このNotebookで構築するCoursewareHubでは[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)の[GakuNin mAP](https://meatwiki.nii.ac.jp/confluence/display/gakuninmappublic/Home)を利用して利用者のグループ管理を行います。

CoursewareHubの利用対象者からなるグループを、学認クラウドゲートウェイにて作成しておいてください。

### UnitGroup名

CoursewareHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

### ホスト名

CoursewareHubのホスト名(FQDN)を指定します。ここで指定したホスト名は、利用者からの入り口となる `auth-proxy` コンテナに割り当てられます。

In [ ]:
# (例)
# master_fqdn = 'hub.example.org'

master_fqdn = 

### 管理者情報の設定

管理者として登録するメールアドレスを指定します。CoursewareHubに管理者としてログインするにはここで指定したメールアドレスをログイン画面で入力します。

> このNotebookでは、CoursewareHubの学認への登録がまだ行われていないことを想定して、管理者をローカルユーザとして作成します。

In [ ]:
# (例)
# teacher_email = 'admin@example.org'

teacher_email = 

管理者のパスワードを入力してください。ここで指定するパスワードはローカルユーザとしてログインする際に入力するパスワードとなります。

> 学認への登録を行うまでは、学認のIdPを利用したログインができません。そのためローカルユーザとして登録するパスワードをここでは指定します。

In [ ]:
from getpass import getpass
teacher_password = getpass()

### 学認フェデレーション

CoursewareHubと連携する学認フェデレーションに関するパラメータを設定します。

次のセルで参加するフェデレーションを指定してください。

In [ ]:
# (例)
# federation = 'production'  # 運用フェデレーション
# federation = 'test'        # テストフェデレーション

federation = 

参加するフェデレーションに応じたサーバとの連携を行います。各フェデレーションのホスト名を以下の表に示します。


| | 学認DS(Discovery Service) | クラウドゲートウェイ |
|-|--|--|
|運用フェデレーション|`ds.gakunin.nii.ac.jp`|`cg.gakunin.jp`|
|テストフェデレーション|`test-ds.gakunin.nii.ac.jp`|`sptest.cg.gakunin.jp`|


上記の表に示した以外のサーバを指定する場合は、次のセルのコメントを外してサーバのホスト名を設定してください。

In [ ]:
# ds_fqdn = 'ds.gakunin.nii.ac.jp'     # 学認DS(Discovery Service)
# cg_fqdn = 'cg.gakunin.jp'            # クラウドゲートウェイ

### mAPのグループを指定する

CoursewareHubの利用を許可するユーザを、学認mAPのグループにより指定します。

CoursewareHubの利用を許可するグループのリストを次のセルで指定してください。指定する値は学認 mAP のグループIDの値となります。

In [ ]:
# (例)
# cg_groups = [
#    'group1-id',
#    'group2-id',
#]

cg_groups = [
    
]

mAPから渡されるグループ情報のプレフィックスを指定します。利用しているフェデレーションによって異なる値を指定する必要があります。

* 運用フェデレーション
    - `https://cg.gakunin.jp/gr/`
* テストフェデレーション
    - `https://sptest.cg.gakunin.jp/gr/`

In [ ]:
if federation == 'production':
    cg_group_prefix = 'https://cg.gakunin.jp/gr/'        # 運用フェデレーション
elif federation == 'test':
    cg_group_prefix = 'https://sptest.cg.gakunin.jp/gr/' # テストフェデレーション
else:
    raise RuntimeError("federationが設定されていません")
    
print(cg_group_prefix)

### リソース制限の設定

各ユーザが利用するコンテナのリソース制限に関する設定を行います。

#### リソース制限の設定方法

CoursewareHubではユーザの役割に応じて利用方法が異なることがあります。例えば講師権限ユーザーは採点を行う場合などに複数のNotebookを同時に開くことがあります。そのような利用を行う場合、一般の受講者ユーザよりも多くのリソースが必要となります。また、講師権限を与えないが採点の補助をして欲しいTA(Teaching Assistant)といった役割のユーザーがいる場合があります。その場合、一般の受講者ユーザーとは異なるリソース制限設定を行うことが考えられます。

これらのことに対応するためにCoursewareHubでは、権限もしくは所属グループに応じたリソース制限の設定を行うことができます。

リソース制限の設定はYAMLファイルで行います。YAMLファイルの記述例を以下に示します。

```yaml
groups:
    student:
        mem_limit: 1G
        cpu_limit: 1.0
        priority: 10
    teaching-assistant:
        mem_limit: 2G
        cpu_limit: 2.0
        cpu_guarantee: 1G
        cpu_guarantee: 0.5
        priority: 1
admin:
    mem_limit: 5G
default:
    mem_limit: 1G
    cpu_limit: 2.0
    mem_guarantee: 1G
    cpu_guarantee: 0.5
```

上の例では `student`グループ、`teaching-assistant`グループ、講師権限ユーザ(`admin`)、それ以外のユーザ（デフォルト設定）についてリソース制限の設定を行っています。複数のグループに所属するユーザについては、グループの`priority`が小さいほうのグループの設定が優先されます。上記の例では`student`グループ、`teaching-assistant`グループの両方に属するユーザは `priority`が`1`となっている`teaching-assistant`グループの設定が優先されます。

コンテナに対するリソース制限設定として以下の４つの属性を指定することができます。

* [mem_guarantee](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.Spawner.mem_guarantee)
    - コンテナの使用が保証されるメモリサイズの下限
* [mem_limit](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.Spawner.mem_limit)
    - コンテナが使用可能なメモリのサイズの上限
* [cpu_guarantee](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.LocalProcessSpawner.cpu_guarantee)
    - コンテナの使用が保証される CPU 使用率の下限
* [cpu_limit](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.LocalProcessSpawner.cpu_limit)
    - コンテナが使用可能な CPU 使用率の上限

JupyterHub にはユーザーグループを扱う仕組みがありますが、それを管理するためのユーザーインターフェースは存在しません。そのため、講師権限ユーザー向けにグループ操作を行うための Notebook を用意しました。

* 83_AddUsersToGroup.ipynb
    - ユーザーを指定したグループに追加する Notebook
* 84_RemoveUsersFromGroup.ipynb
    - ユーザーを指定したグループから削除する Notebook
* 85_ShowUserGroups.ipynb
    - ユーザーの属するグループを確認する Notebook

構築したCoursewareHub環境において、講師権限ユーザーに提供される一連のNotebookのなかに上記のグループ管理用 Notebook が含まれています。

#### リソース制限の設定を行うYAMLファイルを作成する

前節で説明したYAMLファイルを作成します。

リソースファイルを作成するディレクトリを作成します。

In [ ]:
from tempfile import mkdtemp
from pathlib import Path
rsc_pdir = Path('./rsc').absolute()
rsc_pdir.mkdir(exist_ok=True)
rsc_dir = Path(mkdtemp(dir=rsc_pdir))

YAMLファイルを作成します。

リソース制限の設定に対応する内容に次のセルを変更してください。その後、実行するとYAMLファイルが作成されます。

In [ ]:
%%writefile {rsc_dir}/resource.yaml
groups:
    group1:
        mem_limit: 1G
        cpu_limit: 1.0
        priority: 20
admin:
    mem_limit: 5G
default:
    mem_limit: 1G
    cpu_limit: 2.0
    mem_guarantee: 1G
    cpu_guarantee: 0.5

YAMLファイルの記述内容が妥当であるかをチェックします。

次のセルが正常に実行できることを確認してください。実行結果がエラーとなった場合はYAMLファイルの記述内容に問題があります。上のセルを unfreeze して記述内容を修正して、再実行してください。

In [ ]:
import jsonschema
import json
import yaml

!ansible -c local {target_hub} -m get_url -a \
    'url=https://raw.githubusercontent.com/NII-cloud-operation/CoursewareHub-LC_platform/master/jupyterhub/resources-schema.json \
    dest={rsc_dir}/resources-schema.json'
with (rsc_dir / 'resources-schema.json').open() as f:
    resources_config_schema = json.load(f)

with (rsc_dir / 'resource.yaml').open() as f:
    resources_config = yaml.load(f, Loader=yaml.SafeLoader)
    jsonschema.validate(resources_config, resources_config_schema)

print(json.dumps(resources_config, indent=2))

### JupyterHub

JupyterHubコンテナに設定する環境変数を指定します。

設定できる環境変数の説明を以下に示します。

* `CONCURRENT_SPAWN_LIMIT`
    - ユーザーのNotebookサーバーの並列起動処理数の上限
    - サーバー起動待ちの数が上限を超えると、新たな起動が拒否される
* `SPAWNER_CONSTRAINTS`
    - ユーザーのNotebookサーバーコンテナの起動ノードの制約条件
    - https://docs.docker.com/engine/swarm/services/#placement-constraints の --constraintの値を指定する
    - 複数の値を指定する場合は `;` でつなげる
    - `node.role==worker`を指定することを推奨する
        + ⇒single-user serverコンテナは workerノードで実行され、masterノードではシステムコンテナのみが実行されるようになる
* `SPAWNER_HTTP_TIMEOUT`
    - JupyterHubが、ユーザーのNotebookサーバーへのHTTPアクセスが可能になるまで待つタイムアウト時間（秒） 
* `SPAWNER_START_TIMEOUT`
    - JupyterHubが、ユーザーのNotebookサーバーコンテナの起動を待つタイムアウト時間（秒）
* `CULL_SERVER`
    - `yes`: ユーザーのNotebookサーバーのculling機能（アイドルになったサーバーの停止）を有効にする
    - `no`: ユーザーのNotebookサーバーは自動で停止されない
    - デフォルトは `no`
* `CULL_SERVER_IDLE_TIMEOUT`
    - culling機能で停止されるNotebookサーバーのアイドル時間（秒）
    - この時間を越えてアイドル状態のサーバーは停止される
    - デフォルト値は `600`   
* `CULL_SERVER_MAX_AGE`
    - ユーザーNotebookサーバーの最大寿命（秒）
    - アクティブでも停止される
    - デフォルト、または`0`ではアクティブなサーバーは停止されない
* `CULL_SERVER_EVERY`
    - サーバーのアイドル状態のチェック間隔(秒)
    - チェックとチェックの間では、アイドル状態になっても停止されない
* `ADMIN_ACCESS`
    - `yes`または`1`を設定すると、講師がAdminコントロールパネルから他のユーザーのNotebookサーバーにアクセスできるようになる
    - デフォルトで有効

設定する環境変数名とその設定値を、次のセルの`dict`変数`jupyterhub_params`のキー、バリューに指定してください。

In [ ]:
# (例)
# jupyterhub_params = {
#     'SPAWNER_CONSTRAINTS': 'node.role==worker',  # 起動ノードの制約条件
# }

jupyterhub_params = {
    'SPAWNER_CONSTRAINTS': 'node.role==worker',
}

### single-user Jupyter notebook server

JupyterHubが起動する single-user [Jupyter notebook](https://jupyter-notebook.readthedocs.io/) serverのコンテナイメージを指定します。

以下のようなものを指定できます。

* [niicloudoperation/notebook](https://github.com/NII-cloud-operation/Jupyter-LC_docker)
    - JupyterNotebookに*Literate Computing for Reproducible Infrastructure*のためのツールを追加したもの
* harbor.vcloud.nii.ac.jp/vcp/coursewarehub:singleuser-disable-run-through
    - `niicloudoperation/notebook`に対して[Jupyter-LC_run_through](https://github.com/NII-cloud-operation/Jupyter-LC_run_through)の機能などを無効にしたもの

In [ ]:
# (例)
# singleuser_image = 'niicloudoperation/notebook'
# singleuser_image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:singleuser-disable-run-through'

singleuser_image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:singleuser-disable-run-through'

### データベース

CoursewareHubのデータを保存するデータベースに関するパラメータを指定します。

CoursewareHubが利用するデータベースの名前を指定してください。

In [ ]:
# (例)
# db_name = 'jupyterhub'

db_name = 'jupyterhub'

データベースのユーザ名を指定してください。

In [ ]:
# (例)
# db_user = 'jhauth'

db_user = 'jhauth'

データベースのパスワードを指定してください。

In [ ]:
from getpass import getpass
db_password = getpass()

### オーバーレイネットワークの指定

CoursewareHubのコンテナはDocker Swarmのオーバーレイネットワークで実行されます。オーバーレイネットワークに割り当てるサブネットを指定します。

> 基本的にはデフォルト値からの変更は不要ですが、VCノードに割り当てられているIPアドレスと範囲が重複している場合は他のサブネットに変更して下さい。

In [ ]:
# (例)
# cousewarehub_backend = '10.1.0.0/20'

cousewarehub_backend = '10.1.0.0/20'

指定されたサブネットがVCノードに割り当てられているIPアドレスと重なっていないことをチェックします。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ansible-playbook -v -e cousewarehub_backend={cousewarehub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

### パラメータの保存

ここまで指定したパラメータのうち、ansible playbookや他のNotebookでも参照する値を ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

gvars.update({
    'master_fqdn': master_fqdn,
    'rsc_yml': f'{rsc_dir}/resource.yaml',
    'jupyterhub_params': jupyterhub_params,
    'singleuser_image': singleuser_image,
    'db_name': db_name,
    'db_user': db_user,
    'db_password': db_password,
    'cousewarehub_backend': cousewarehub_backend,
    'cg_groups': [cg_group_prefix + x for x in cg_groups],
    'federation': federation,
})
if 'ds_fqdn' in vars():
    gvars['ds_fqdn'] = ds_fqdn
if 'cg_fqdn' in vars():
    gvars['cg_fqdn'] = ds_fqdn

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

## CoursewareHubのセットアップ

CoursewareHubの構成要素となる、各コンテナのセットアップを行います。

### auth-proxy

#### 証明書の配置

##### サーバ証明書

auth-proxyコンテナで使用するサーバ証明書の配置を確認します。

サーバ証明書を配置するディレクトリを作成します。

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}} state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/certs state=directory owner={{{{ansible_user}}}}'

作成したディレクトリ`/srv/cwh/certs`に証明書、秘密鍵を配置してください。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから **managerノードに ssh でログインして操作を行ってください**。

配置する証明書などのファイル名は以下のようにしてください。

* サーバ証明書
    - `/srv/cwh/certs/auth-proxy.cer`
* サーバ証明書と中間CA証明書を連結したもの
    - `/srv/cwh/certs/auth-proxy.chained.cer`
* 秘密鍵
    - `/srv/cwh/certs/auth-proxy.key`
    - パスフレーズを無しにする

sshでログインする manager ノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`vcp`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することで manager ノードにsshでログインできます。

In [ ]:
!ansible {target_hub} -m debug -a 'var=vc_ipaddress'

**証明書などの配置を行った後に、これ以降の操作を行ってください。**

証明書が配置されていることを確認します。managerノードのサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/auth-proxy.cer'

秘密鍵の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl rsa -noout -text -in  {{{{certs_dir}}}}/auth-proxy.key'

中間CA証明書を連結したサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/auth-proxy.chained.cer'

証明書の owner, group などの値を利用環境に合わせて以下のように設定します。

* owner: 33
* group: 33

この設定はコンテナ内では以下のように設定したことに相当します。

* owner: `www-data`
* group: `www-data`

In [ ]:
cert_owner = 33
cert_group = 33
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}} owner={cert_owner} \
    group={cert_group} state=directory'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/auth-proxy.cer \
    owner={cert_owner} group={cert_group}'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/auth-proxy.chained.cer \
    owner={cert_owner} group={cert_group}'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/auth-proxy.key \
    owner={cert_owner} group={cert_group} mode=0600'

##### メタデータ署名証明書の配置

メタデータの署名を検証するための証明書を配置します。

証明書を配布しているURLを指定してください。

In [ ]:
# 運用フェデレーション
# metadata_signer_url = 'https://metadata.gakunin.nii.ac.jp/gakunin-signer-2017.cer'
#
# テストフェデレーション
# metadata_signer_url = 'https://metadata.gakunin.nii.ac.jp/gakunin-test-signer-2020.cer'

metadata_signer_url =

指定されたURLの証明書を配置します。

In [ ]:
!ansible {target_hub} -b -m get_url -a \
    'url={metadata_signer_url} dest={{{{certs_dir}}}}/gakunin-signer.cer \
    owner={cert_owner} group={cert_group} mode=0644'

配置した証明書のFingerprintを表示してみます。

参考のため、証明書の fingerprintを記載しているURLを以下に示します。

* 運用フェデレーション
    - https://meatwiki.nii.ac.jp/confluence/display/GakuNinShibInstall/signer
* テストフェデレーション
    - https://www.gakunin.jp/join/test/rule

In [ ]:
!ansible {target_hub} -a 'chdir={{{{base_dir}}}}/certs \
    openssl x509 -in gakunin-signer.cer -sha256 -fingerprint -noout'

#### メタデータ

SAMLのメタデータを配置するディレクトリを用意します。

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/metadata state=directory owner=33 group=33'

##### SP検証環境のメタデータ（テストフェデレーション）

テストフェデレーションのSP検証環境メタデータを取得している場合は CoursewareHub環境への配置を行います。

> SP検証環境のメタデータは利用申請の際に送付されてくるので、通常はこの時点でメタデータファイルがありません。そのため**この節は実行せずにスキップしてください**。通常は、次に実行する「331-CoursewareHubの登録を学認へ申請する」のNotebookのなかで SP検証環境のメタデータを登録します。
>
> この節は、既に学認への登録を済ませている CoursewareHub を再構築する場合などに実行することを想定しています。

SP検証環境のメタデータを、このNotebook環境に配置してそのファイル名を次のセルに設定してください。

In [ ]:
# (例)
# sptest_metadata = './sptestcgidp-metadata.xml'

sptest_metadata = 

SP検証環境のメタデータを配置します。

In [ ]:
if 'sptest_metadata' in vars():
    !ansible {target_hub} -b -m copy -a \
        'src={sptest_metadata} dest={{{{base_dir}}}}/metadata/cgidp-metadata.xml \
        owner=33 group=33'

#### auth-proxyコンテナのセットアップ

サーバ証明書以外に 
auth-proxyコンテナで必要となるファイルを準備する Ansible Playbook を実行します。

この節で実行する Playbook では以下の処理を行います。
* SimpleSAMLphp のcron実行のためのランダムキーの作成
* コンテナイメージの取得

ここで取得するauth-proxyのコンテナイメージは[NII-cloud-operation/CoursewareHub-LC_platform](https://github.com/NII-cloud-operation/CoursewareHub-LC_platform)の `auth-proxy-fed/`ディレクトリをVCP向けにカスタマイズして [VCPのコンテナレジストリ](https://harbor.vcloud.nii.ac.jp/)に格納したものです。VCPでカスタマイズした部分のソースは [./docker/app/auth-proxy-fed](docker/app/auth-proxy-fed/)にあります。

まず、実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-auth-proxy.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-auth-proxy.yml

auth-proxy コンテナイメージを取得したことを確認します。

In [ ]:
!ansible {target_hub} -m shell -a 'docker images | grep auth-proxy'

auth-proxyコンテナのためにセットアップしたディレクトリの状態を確認します。

In [ ]:
!ansible {target_hub} -a 'tree {{{{auth_proxy_dir}}}}'

### JupyterHub

#### restuserのインストール

JupyterHubコンテナからホスト環境のローカルユーザ情報を取得するために利用する[restuser](https://github.com/minrk/restuser)をインストールします。

ここでは Ansible Playbookを実行することで restuser のインストールを行います。実行する Playbook は
[NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy](https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy) の `roles/restuser/` にあるものをもとにしています。

実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/install-restuser.yml || true

実際にrestuserのインストールを行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/install-restuser.yml

インストールされたファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree /srv/restuser'

restuserはホスト環境のサービスとして実行します。インストールしたrestuserサービスの状態を確認します。次のセルの出力結果に`Active: active (running)` と表示されることを確認してください。

In [ ]:
!ansible {target_hub} -b -a 'systemctl status restuser'

`restuser`によってユーザ情報が取得できることを確認します。HTTPの応答がOK(200)となり、以下の情報が取得できることを確認してください。

```
{"name": "vcp", "dir": "/home/vcp", "shell": "/bin/bash", "uid": 1000, "gid": 1000}
```

In [ ]:
!ansible {target_hub} -b -m shell \
    -a 'echo -e "POST /{{{{ansible_user}}}} HTTP/1.0\r\n" | nc -U /var/run/restuser.sock'

#### JupyterHubコンテナに関するファイルを準備する

JupyterHubコンテナを実行するために必要となるファイルを準備する Ansible Playbook を実行します。

Playbook では以下の処理を行います。

* コンテナイメージの取得
* ロゴファイルの配置


ここで取得するJupyterHubのコンテナイメージは[NII-cloud-operation/CoursewareHub-LC_platform](https://github.com/NII-cloud-operation/CoursewareHub-LC_platform)の `jupyterhub/`ディレクトリをビルドして [VCPのコンテナレジストリ](https://harbor.vcloud.nii.ac.jp/)に格納したものです。

まず、実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

> ドライランではロゴファイルを配置するディレクトリが作成されないため、ファイルの配置でエラーとなりますがこの時点では問題ありません。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-jupyterhub.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-jupyterhub.yml

JupyterHubのコンテナイメージが取得できたことを確認します。

In [ ]:
!ansible {target_hub} -a \
    'docker images -f label=org.jupyter.service=jupyterhub'

JupyterHubコンテナのためにセットアップしたディレクトリの状態を確認します。

In [ ]:
!ansible {target_hub} -a 'tree {{{{jupyterhub_dir}}}}'

### PostgreSQL

PostgreSQLコンテナに関するファイルを準備する Ansible Playbook を実行します。

この節で実行する Playbook では以下の処理を行います。
* PostgreSQLのデータを格納するディレクトリの作成
* 初期設定SQLファイルの配置
* コンテナイメージの取得

実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-postgres.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-postgres.yml

PostgreSQLコンテナのためにセットアップしたディレクトリの状態を確認します。

In [ ]:
!ansible {target_hub} -b -a 'tree {{{{postgres_dir}}}}'

### single-user Jupyter Notebook server

各VCノードにsingle-user serverのコンテナイメージを取得します。

In [ ]:
!ansible {ugroup_name} -a 'docker pull {{{{singleuser_image}}}}'

JupyterHubからコンテナとして起動する際に指定するタグ名をコンテナイメージに設定します。

In [ ]:
!ansible {ugroup_name} -a \
    'docker tag {{{{singleuser_image}}}} niicloudoperation/jupyterhub-singleuser'

各VCノードのコンテナイメージ一覧を確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a 'docker images | \
    grep -e "niicloudoperation/jupyterhub-singleuser"'

## コンテナの起動

### docker-compose.yml の配置

複数のコンテナに関する設定をまとめて扱うために `docker-compose.yml` を利用します。ここまでに指定されたパラメータに応じた`docker-compose.yml`を生成し、構築環境に配置します。

まずは、チェックモードで確認を行います。

In [ ]:
!ansible {target_hub} -CDv -m template \
    -a 'src=template/docker-compose.yml dest={{{{base_dir}}}} backup=yes'

実際に`docker-compose.yml`の配置を行います。

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=template/docker-compose.yml dest={{{{base_dir}}}} backup=yes'

### コンテナの起動

コンテナを起動します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{base_dir}}}} \
    docker stack deploy -c docker-compose.yml {{{{ugroup_name}}}}'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker stack ps {{{{ugroup_name}}}}'

全てのコンテナが起動するまで待ち合わせを行います。

> 次のセルの実行結果がエラーとなる場合は、その後のセルを実行してコンテナの実行状況やログを確認してください。

In [ ]:
import time
import sys

out = !ansible {target_hub} -c local -a 'echo "{{{{vc_ipaddress}}}}"'
ip_addr = out[1]
for retry in range(18):
    try:
        !ansible {target_hub} -a \
            'curl -s -k -I -f --resolve "{{{{master_fqdn}}}}:443:{ip_addr}" \
            https://{master_fqdn}'
        break
    except RuntimeError:
        print('retry', file=sys.stderr)
        time.sleep(10)
else:
    !ansible {target_hub} -a \
            'curl -s -k -I -f --resolve "{{{{master_fqdn}}}}:443:{ip_addr}" \
            https://{master_fqdn}'

起動後の状態を確認します。
> コンテナが起動に失敗し何度も再起動されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker stack ps {{{{ugroup_name}}}}'

`postgres` コンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_postgres'

`jupyterhub` コンテナのログを表示してみます。

> PostgreSQLに接続できないなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_jupyterhub'

`auth-proxy` コンテナのログを表示してみます。

> 証明書設定に誤りがあるなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_auth-proxy'

## 管理者の追加

### Systemユーザの作成

CoursewareHubではメールアドレスから一定のルールで導き出された名前を、ローカルユーザ名として利用します。管理ユーザのローカルユーザ名を確認します。

In [ ]:
import hashlib
import re

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except alphabetic
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h
    return result

In [ ]:
teacher_id = get_username_from_mail_address(teacher_email)
print(teacher_id)

ホームディレクトリの親ディレクトリを作成します。

In [ ]:
!ansible {target_hub} -b -m file -a 'path=/jupyter/users state=directory'

管理者のホームディレクトリを変数に設定しておきます。

In [ ]:
teacher_homedir = f'/jupyter/users/{teacher_id}'
print(teacher_homedir)

`manager`ノードでユーザを作成します。

In [ ]:
!ansible {target_hub} -b -m user -a 'name={teacher_id} home={teacher_homedir}'

ホームディレクトリが作成されていることを確認します。

In [ ]:
!ansible {target_hub} -b -a 'ls -la {teacher_homedir}'

`worker`ノードにも同じ名前のユーザを作成します。ホームディレクトリはNFSになるので、`manager` のUID/GIDと同じ値でユーザを作成します。

まず、`manager` での UID/GID の値を確認します。

In [ ]:
lines = !ansible {target_hub} -a 'id -u {teacher_id}'
teacher_uid = lines[1]
lines = !ansible {target_hub} -a 'id -g {teacher_id}'
teacher_gid = lines[1]
lines = !ansible {target_hub} -a 'id -g -n {teacher_id}'
teacher_group = lines[1]
(teacher_uid, teacher_gid, teacher_group)

`worker`ノードでグループを作成します。

> GID を指定するので、まずグループを作成します。

In [ ]:
!ansible {target_nodes} -b -m group -a 'name={teacher_group} gid={teacher_gid}'

`worker`ノードでユーザを作成します。

In [ ]:
!ansible {target_nodes} -b -m file \
    -a 'path=/jupyter/users state=directory'

In [ ]:
!ansible {target_nodes} -b -m user \
    -a 'name={teacher_id} uid={teacher_uid} group={teacher_group} \
        home={teacher_homedir}'

ユーザが作成されたことを確認します。

In [ ]:
!ansible {target_hub} -a 'id {teacher_id}'
!ansible {target_nodes} -a 'id {teacher_id}'

### Prepare contents directory

コンテンツを格納するディレクトリ `info`, `textbook` を準備します。

In [ ]:
for x in ['info', 'textbook']:
    !ansible {target_hub} -b -m file \
        -a 'path={teacher_homedir}/{x} state=directory \
            owner={teacher_uid} group={teacher_group} mode=0777'

### Create SSH key and register

JupyterHubを構成するマシンへログインするための鍵の生成と登録を行います。

SSHの鍵ペアを作成します。

In [ ]:
!ansible {target_hub} -b -a 'creates={teacher_homedir}/.ssh/id_rsa \
    sudo -u {teacher_id} \
    ssh-keygen -N "" -f {teacher_homedir}/.ssh/id_rsa'

鍵ファイルが作成されたことを確認します。

In [ ]:
!ansible {target_hub} -b -m shell -a 'ls -l {teacher_homedir}/.ssh/id_rsa*'

作成した公開鍵を `authorized_keys` に登録します。まず、公開鍵の値を取得します。

In [ ]:
lines = !ansible {target_hub} -b -a 'cat {teacher_homedir}/.ssh/id_rsa.pub'
pubkey = lines[1]
print(pubkey)

`authorized_keys`に登録します。

In [ ]:
!ansible {target_hub} -b -m authorized_key -a 'user={teacher_id} key="{pubkey}"'

ユーザーのホームディレクトリに不適切なpermissionが設定されているとsshの鍵認証に失敗するので、妥当な値が設定されていることを保証しておきます。

In [ ]:
!ansible {target_hub} -b -m file \
    -a 'path={teacher_homedir} mode="0755" \
        owner={teacher_id} group={teacher_group}'

### Grant sudo

JupyterHubを構成するマシン上でのsudo権限を与える設定ファイルを配置します。

事前のチェックを行います。

In [ ]:
!ansible {target_hub} -CD -b -m lineinfile \
    -a 'dest=/etc/sudoers.d/{teacher_id} create=yes\
        line="{teacher_id} ALL=(ALL) NOPASSWD: ALL"'

実際に設定ファイルの配置を行います。

In [ ]:
!ansible {target_hub} -b -m lineinfile \
    -a 'dest=/etc/sudoers.d/{teacher_id} create=yes\
        line="{teacher_id} ALL=(ALL) NOPASSWD: ALL"'

### Set ansible inventory

JupyterHubを構成するマシンを操作するためのインベントリを配布します。

まずは、インベントリ配布するための playbook をチェックモード実行します。

In [ ]:
!ansible-playbook -CDv -l {target_hub} \
    -e teacher_id={teacher_id} -e teacher_homedir={teacher_homedir} \
    -e target_hub={target_hub} -e target_nodes={target_nodes} \
    playbooks/deploy-inventory.yml \
    || true

実際にインベントリを構築環境の`~/ansible/inventory`に配布します。

In [ ]:
!ansible-playbook -Dv -l {target_hub} \
    -e teacher_id={teacher_id} -e teacher_homedir={teacher_homedir} \
    -e target_hub={target_hub} -e target_nodes={target_nodes} \
    playbooks/deploy-inventory.yml

### JupyterHubユーザの作成


初回のユーザー作成ではJupyterHub APIが使用できないので、直接DBを変更してユーザーを登録します。

まず、JupyterHubのユーザテーブルにユーザを追加するための SQL ファイルを作成します。

In [ ]:
import random
from datetime import datetime
from tempfile import TemporaryDirectory

cookie_id = ''.join(random.choices("0123456789abcdef", k=32))
with TemporaryDirectory() as tmp_dir:
    sql_file = Path(tmp_dir) / ('create_user_{:%Y%m%d_%H%M%S}.sql'.format(datetime.now()))
    with sql_file.open(mode='w') as f:
        f.write(f'''
INSERT INTO users (name, admin, cookie_id, last_activity)
  VALUES ('{teacher_id}', TRUE, '{cookie_id}', '{datetime.now().isoformat()}');
''')
    !cat {str(sql_file)}
    !ansible {target_hub} -b -m copy -a 'src={str(sql_file)} dest=/jupyter/psql/init/'

PostgreSQLコンテナのコンテナIDと実行しているホストのIPアドレスを取得します。

In [ ]:
lines = !ansible {target_hub} -b -a 'docker service ps {{{{ugroup_name}}}}_postgres -q'
sid = lines[1]

lines = !ansible {target_hub} -b -a \
    'docker inspect --format "{{% raw %}} {{{{.NodeID}}}} {{{{.Status.ContainerStatus.ContainerID}}}} {{% endraw %}}" {sid}'
nodeid, cid = lines[1].split()
print(cid)

lines = !ansible {target_hub} -b -a \
    'docker node inspect --format "{{% raw %}}{{{{.Status.Addr}}}} {{% endraw %}}" {nodeid}'
target_ip = lines[1].split()[0]
print(target_ip)

SQLファイルを実行します。

In [ ]:
!ansible {target_ip} -b -a 'docker exec -i {cid} \
    psql -d {{{{db_name}}}} -U {{{{db_user}}}} -f /docker-entrypoint-initdb.d/{sql_file.name}'

ユーザが登録されたことを確認します。

In [ ]:
!ansible {target_ip} -a 'docker exec -i {cid} \
    psql -d {{{{db_name}}}} -U {{{{db_user}}}} -c "SELECT * FROM users"'

ローカルユーザーのテーブルにも登録します。 SQLファイルを作成します。

In [ ]:
from crypt import crypt

with TemporaryDirectory() as tmp_dir:
    sql2_file = Path(tmp_dir) / ('create_localuser_{:%Y%m%d_%H%M%S}.sql'.format(datetime.now()))
    with sql2_file.open(mode='w') as f:
        f.write(f'''
INSERT INTO local_users VALUES(
  nextval('local_users_id_seq'),
  '{teacher_id}',
  '{crypt(teacher_password)}',
  '{teacher_email}'
);
''')
    !ansible {target_hub} -b -m template -a 'src={str(sql2_file)} dest=/jupyter/psql/init/'
    !ansible {target_hub} -b -a 'cat /jupyter/psql/init/{str(sql2_file.name)}'

SQLファイルを実行します。

In [ ]:
!ansible {target_ip} -b -a 'docker exec -i {cid} \
    psql -d {{{{db_name}}}} -U {{{{db_user}}}} -f /docker-entrypoint-initdb.d/{sql2_file.name}'

ユーザ登録のために一時的に作成したSQLファイルを削除します。

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path=/jupyter/psql/init/{sql_file.name} state=absent'
!ansible {target_hub} -b -m file -a \
    'path=/jupyter/psql/init/{sql2_file.name} state=absent'

## コンテンツの配備の準備

CoursewareHubのコンテンツを格納するディレクトリを作成し、コンテンツの配置を行うNotebookを管理者のホームディレクトリに配置します。

実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv -e teacher_id={teacher_id} playbooks/manage-tools.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -e teacher_id={teacher_id} playbooks/manage-tools.yml

## SimpleSAMLphpの設定ファイルを更新する

構築環境のメタデータに設定されている技術的問い合わせ先などの情報を更新するために SimpleSAMLphpの設定ファイルを編集します。

まず、構築環境の auth-proxyコンテナからSimpleSAMLphpの`config.php`をローカル環境に取得します。

In [ ]:
Path('./tmp').mkdir(exist_ok=True)
work_dir = Path(mkdtemp(dir='./tmp')).absolute()
simplesamlphp_config = work_dir / 'config.php'
!ansible-playbook -v -e simplesamlphp_config={simplesamlphp_config} -l {target_hub} \
    playbooks/fetch-simplesaml-config.yml

次のセルの実行結果に表示されるリンクをクリックすることで、取得した `config.php`の内容を編集することができます。必要な項目を編集してください。技術的問い合わせ先を修正するには以下の項目を変更してください。

* `technicalcontact_name`
* `technicalcontact_email`

> ファイルの編集後にはCtrl-Sなどで編集結果を保存してください。

またSimpleSAMLphpの管理者パスワードを設定する以下の項目の設定を変更してください。

* `auth.adminpassword`

In [ ]:
from notebook import notebookapp
from IPython.display import HTML
nb_conf = list(notebookapp.list_running_servers())[0]
p = Path(nb_conf['base_url']) / 'edit' / simplesamlphp_config.relative_to(nb_conf['notebook_dir'])
HTML(f'<a href={p} target="_blank">{p.name}</a>')

ローカル環境で編集した設定ファイル`config.php`を実行環境のauth-proxyコンテナに配置します。

In [ ]:
!ansible-playbook -v -e simplesamlphp_config={simplesamlphp_config} -l {target_hub} \
    playbooks/deploy-simplesaml-config.yml

次のセルを実行すると表示されるリンク先を表示するとメタデータが表示されます。技術的問い合わせ先などの値が更新されていることを確認してください。

In [ ]:
print(f'https://{master_fqdn}/simplesaml/module.php/saml/sp/metadata.php/default-sp?output=xhtml')

作業ディレクトリを削除します。

In [ ]:
!rm -rf {work_dir}

## CoursewareHubにアクセスする

構築環境にアクセスして、正しく動作していることを確認してください。

次のセルを実行すると、構築したCoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{master_fqdn}')

このNotebookで管理者として登録したユーザのsingle-userサーバの環境には、CoursewareHubのコンテンツを配備するためのNotebook「CoursewareHubコンテンツの配備.ipynb」が用意されています。管理者としてCoursewareHubにログインした後に実行してください。

> ログイン画面に「学認フェデレーションへ」とのリンクが表示されていますが「331-CoursewareHubの登録を学認へ申請する.ipynb」を実行するまでは、学認IdPを利用したログインは行えません。